In [14]:
import requests
import pandas as pd

# Define your FRED API key
API_KEY = "dc6a0313beabf8a2845ed521ddb56d7b"

# Define the FRED API endpoint for FOMC meeting dates
FRED_ENDPOINT = 'https://api.stlouisfed.org/fred/series/observations'

# Parameters for the API request
params = {
    'series_id': 'FEDFUNDS',  # This series represents the federal funds rate, which is influenced by FOMC meetings
    'api_key': API_KEY,
    'file_type': 'json'
}

# Make the API request
response = requests.get(FRED_ENDPOINT, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    
    # Extract observation dates
    observations = data['observations']
    
    
    # Create a DataFrame
    #df = pd.DataFrame(dates, columns=['Date'])
    df = pd.DataFrame(observations)
    df = df[['date', 'value']]
    df.columns = ['date', 'ffr']
    df['date'] = pd.to_datetime(df['date'])
    # Print the DataFrame
    print(df)
else:
    print(f"Error fetching data: {response.status_code}")

# Optionally, save the data to a CSV file
#df.to_csv('fomc_meeting_dates.csv', index=False)


          date   ffr
0   1954-07-01  0.80
1   1954-08-01  1.22
2   1954-09-01  1.07
3   1954-10-01  0.85
4   1954-11-01  0.83
..         ...   ...
835 2024-02-01  5.33
836 2024-03-01  5.33
837 2024-04-01  5.33
838 2024-05-01  5.33
839 2024-06-01  5.33

[840 rows x 2 columns]


In [235]:
def complete_month(month_abbr):
  """Completes a month name from its abbreviation.

  Args:
    month_abbr: The first three letters of a month name.

  Returns:
    The complete month name if found, otherwise None.
  """

  for month_number, month_name in enumerate(calendar.month_name):
    if month_name.lower().startswith(month_abbr.lower()):
      return month_name
  return None

def month_str_to_num(month_str):
  """Converts a month string to its corresponding number.

  Args:
    month_str: The month name as a string (e.g., "January", "Feb").

  Returns:
    The month number (1-12) if found, otherwise None.
  """

  month_str = month_str.lower()
  for month_num, month_name in enumerate(calendar.month_name):
    if month_name.lower().startswith(month_str):
      return month_num
  return None

In [269]:
from bs4 import BeautifulSoup
import calendar
import datetime
import re 
import requests

year = 2024 # in your code take the today date and extract the year to automate the entire process 

# URL of the FOMC meeting calendar page
url = "https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm"

# Make a GET request to fetch the raw HTML content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the header with the text "2024 FOMC Meetings"
    header = soup.find('h4', string=f"{year} FOMC Meetings")

    parent_div1 = header.find_parent('div')

    main_parent= parent_div1.find_parent('div')

    months = main_parent.find_all('div', re.compile("fomc-meeting__month "))
    days = main_parent.find_all('div', "fomc-meeting__date col-xs-4 col-sm-9 col-md-10 col-lg-1")
    
    dates = {}
    for i in range(len(months)):
        
        month = months[i].text
        
        if '/' in month:
            m1, m2 = month.split("/")
            d1, d2 = days[i].text.split('-')
            
            #m1 = complete_month(m1)
            m2 = complete_month(m2)
            
            #dates[m1] = d1 # just remove this line if not interested in day 1 of the FED 
            dates[m2] = d2
            
            continue
        
        else:
            dates[months[i].text] = days[i].text.split('-')

    #dates = pd.DataFrame(dates, index = [1,2] ,columns = dates.keys())
    
dates  
    

{'January': ['30', '31'],
 'March': ['19', '20*'],
 'May': '1',
 'June': ['11', '12*'],
 'July': ['30', '31'],
 'September': ['17', '18*'],
 'November': ['6', '7'],
 'December': ['17', '18*']}

In [270]:
today = datetime.date.today()
month = calendar.month_name[today.month]
day = today.day
print("today: ", today)
print("Month and Day: ",  (year, month, day ))

today:  2024-08-01
Month and Day:  (2024, 'August', 1)


In [271]:

try:
    days = dates[month]
    print(f'Fed Decision Day on {month}', days, year)
    try:
        #fed_decision_date = datetime.date(year, month_str_to_num(month),int(days[1].replace('*','')))
        fed_decision_date = datetime.date(year, today.month, int(days[1].replace('*','')))
    except IndexError:
        #fed_decision_date = datetime.date(year, month_str_to_num(month),int(days[0].replace('*','')))
        fed_decision_date = datetime.date(year, today.month, int(days[0].replace('*','')))
    
except KeyError:
    print(f'No FED in {month}.')
    next_month = calendar.month_name[(today + datetime.timedelta(days=32)).month]
    days = dates[next_month]
    try:
        fed_decision_date = datetime.date(year, today.month + 1, int(days[1].replace('*','')))
        print(f'Fed Decision on {next_month}', days[1], year)
    except IndexError:
        fed_decision_date = datetime.date(year, today.month + 1, int(days[0].replace('*','')))
        print(f'Fed Decision on {next_month}', days[0], year)

No FED in August.
Fed Decision on September 18* 2024


In [272]:
import datetime

def count_weekdays(start_date, end_date):
  """Counts the number of weekdays between two dates.

  Args:
    start_date: The start date as a datetime.date object.
    end_date: The end date as a datetime.date object.

  Returns:
    The number of weekdays between the two dates.
  """

  delta = end_date - start_date
  total_days = delta.days + 1  # Include both start and end dates

  # Calculate the number of full weeks
  full_weeks, remaining_days = divmod(total_days, 7)
  weekdays = full_weeks * 5  # 5 weekdays per week

  # Handle the remaining days
  start_weekday = start_date.weekday()
  end_weekday = end_date.weekday()

  if start_weekday < 5:  # Start day is not a weekend
    weekdays += min(end_weekday + 1, 5) - start_weekday
  else:
    weekdays += max(0, end_weekday - 4)

  return weekdays

num_weekdays = count_weekdays(today, fed_decision_date)
print("Trading days until FED decision: ", num_weekdays) 
print('Calendar Days until FED decision: ', (fed_decision_date - today).days)


Trading days until FED decision:  35
Calendar Days until FED decision:  48
